In [1]:
#1) Create a summary table of more scenarios
#2) Create a summary of the summary table
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd

In [2]:
ST = pd.read_csv('outputs/SummaryTable_Pilates_TR.csv', index_col = 0).fillna(0)
# plot_and_save_figures('outputs/', ST, ['Baseline_20221224','RH Price 0%','RH Price 12.5%','RH Price 25%','RH Price 50%','RH Price 175%','RH Price 300%','RH Price 500%','RH Price 800%',
#                                       'RH Fleet 12.5%','RH Fleet 25%','RH Fleet 50%','RH Fleet 200%','RH Fleet 400%','RH Fleet 1000%','RH Fleet 2750%'])
Scale = 0.1
is_WC = False
# plot_and_save_figures('outputs/', ST, ['Base 20221224','TR 2018','TR 2019','TR 2020','TR 2021','TR 2022','TR 2023','TR 2024','TR 2018-1','TR 2018-2','TR 2018-3','TR 2018-4','TR 2018-5','TR 2018-6','TR 2018-7','TR 2018-8','TR 2018-9','TR 2018-10','TR 2018-11','TR 2018-12'])
# plot_and_save_figures('outputs/', ST, ['Baseline',
                                       # '1Fl 1Pr 1Flz','5Fl 1Pr 1Flz','5Fl 1Pr 1.6Flz','5Fl 1Pr 2Flz','5Fl 1Pr 4Flz','5Fl 1Pr 10Flz',
                                       # '2Fl 1Pr 1.6Flz','2Fl 1Pr 2Flz','2Fl 1Pr 4Flz','2Fl 1Pr 10Flz',
                                       # '2Fl .47Pr 1Flz','5Fl .47Pr 1Flz','5Fl .47Pr 1.6Flz','5Fl .47Pr 2Flz','5Fl .47Pr 4Flz','5Fl .47Pr 10Flz',
                                       # '2Fl .27Pr 1Flz','5Fl .27Pr 1Flz','5Fl .27Pr 1.6Flz','5Fl .27Pr 2Flz','5Fl .27Pr 4Flz','5Fl .27Pr 10Flz',
                                       # '2Fl .125Pr 1Flz','5Fl .125Pr 1Flz','5Fl .125Pr 1.6Flz','5Fl .125Pr 2Flz','5Fl .125Pr 4Flz','5Fl .125Pr 10Flz',
                                       # '2Fl .0625Pr 1Flz','5Fl .0625Pr 1Flz','5Fl .0625Pr 1.6Flz','5Fl .0625Pr 2Flz','5Fl .0625Pr 4Flz','5Fl .0625Pr 10Flz',
                                       # '1Fl MixPr 1Flz','2Fl MixPr 1Flz','5Fl MixPr 1Flz','5Fl MixPr 1.6Flz','5Fl MixPr 2Flz','5Fl MixPr 4Flz','5Fl MixPr 10Flz'
                                      # ])
plot_and_save_figures('outputs/', ST, [#'BS 2018-1','TR 2018-1','BS 2018-2','TR 2018-2','BS 2018-3','TR 2018-3','BS 2018-4','TR 2018-4','BS 2018-5','TR 2018-5',
                                        'BS 2019-1','TR 2019-1','BS 2019-2','TR 2019-2','BS 2019-3','TR 2019-3','BS 2019-4','TR 2019-4','BS 2019-5','TR 2019-5'
                                      ])



NameError: name 'pd' is not defined

In [ ]:
def collectAllData(outDirectory, year_it,
                  trips, activities, PTs, PtoPTs, LT, PLT, VLT, ST, name):

    
    
    if is_GEO:
        BGs = gpd.read_file('local_files/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')
        trips = addGeometryIdToDataFrame(trips, BGs, 'originX', 'originY', 'startBlockGroup')
        trips = addGeometryIdToDataFrame(trips, BGs, 'destinationX', 'destinationY', 'endBlockGroup')
        activities = addGeometryIdToDataFrame(activities, BGs, 'activityLocationX', 'activityLocationY', 'activityBlockGroup')
        
        print('Add block ID infos')
        block_info = pd.read_csv('local_files/bg_w_geog_labels.csv')
        trips['startBlockGroup'] = trips['startBlockGroup'].fillna(0).astype(int)
        trips['endBlockGroup'] = trips['endBlockGroup'].fillna(0).astype(int)
        activities['activityBlockGroup'] = activities['activityBlockGroup'].fillna(0).astype(int)
        PTs['startBlockGroup'] = PTs['startBlockGroup'].fillna(0).astype(int)
        PTs['endBlockGroup'] = PTs['endBlockGroup'].fillna(0).astype(int)
        PtoPTs['startBlockGroup'] = PtoPTs['startBlockGroup'].fillna(0).astype(int)
        PtoPTs['endBlockGroup'] = PtoPTs['endBlockGroup'].fillna(0).astype(int)
        if is_LT:
            LT['BlockGroup'] = LT['BlockGroup'].fillna(0).astype(int)
            PLT['BlockGroup'] = PLT['BlockGroup'].fillna(0).astype(int)
            VLT['BlockGroup'] = VLT['BlockGroup'].fillna(0).astype(int)
        
        trips = pd.merge(trips, block_info,  how='left',  left_on = 'startBlockGroup',right_on = 'bgid')
        trips = pd.merge(trips, block_info,  how='left',  left_on = 'endBlockGroup',right_on = 'bgid',suffixes = ['_start','_end'])
        activities = pd.merge(activities, block_info,  how='left',  left_on = 'activityBlockGroup',right_on = 'bgid',)
        PTs = pd.merge(PTs, block_info,  how='left',  left_on = 'startBlockGroup',right_on = 'bgid',)
        PTs = pd.merge(PTs, block_info,  how='left',  left_on = 'endBlockGroup',right_on = 'bgid',suffixes = ['_start','_end'])
        PtoPTs = pd.merge(PtoPTs, block_info,  how='left',  left_on = 'startBlockGroup',right_on = 'bgid',)
        PtoPTs = pd.merge(PtoPTs, block_info,  how='left',  left_on = 'endBlockGroup',right_on = 'bgid',suffixes = ['_start','_end'])
        if is_LT:
            LT = pd.merge(LT, block_info,  how='left',  left_on = 'BlockGroup',right_on = 'bgid',)
            PLT = pd.merge(PLT, block_info,  how='left',  left_on = 'BlockGroup',right_on = 'bgid',)
            VLT = pd.merge(VLT, block_info,  how='left',  left_on = 'BlockGroup',right_on = 'bgid',)

    # PTs = addGeometryIdToDataFrame(PTs, BGs, 'startX', 'startY', 'startBlockGroup')
    # PTs = addGeometryIdToDataFrame(PTs, BGs, 'endX', 'endY', 'endBlockGroup')
    PTs.index.set_names('PathTraversalID', inplace=True)
    
    
    print('writing results on', outDirectory+ 'postprocessOutputs/'+year_it)

    trips.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/trips.csv.gz', index=True)
    PTs.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/vehicles.csv.gz', index=True)
    activities.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/activities.csv.gz', index=True)
    PtoPTs.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/personToVehicles.csv.gz', index=True)
    if is_LT:
        LT.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/linkTable.csv.gz', index=True)
        PLT.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/personLinkTable.csv.gz', index=True)
        VLT.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/vehicleLinkTable.csv.gz', index=True) 
    ST.to_csv(outDirectory+ 'postprocessOutputs/'+year_it+'/summaryTable.csv.gz', index=True)
    
    if is_plot:
        plot_and_save_figures(outDirectory+ 'postprocessOutputs/'+year_it+'/figures/', ST, name)
    

In [ ]:
def plot_and_save_figures(fp, ST, name):
    
    colors_std = ['#FF7E72', 
            '#FFAC91',
            '#FFC9B9',
            '#5DC7B6', 
    #         '#79DFCE',
            '#8CD7FF',
            '#69ADF1']

    x_vlines = [0.5, #9.5#6.5,10.5, 16.5, 22.5 , 28.5,34.5 #4.5,
    #             7.5,11.5,15.5,19.5,23.5
               ]
    
    VMT_plot(ST, name, fp, colors_std, x_vlines, Scale)
    VHT_plot(ST, name, fp, colors_std, x_vlines, Scale)
    speed_plot(ST, name, fp, colors_std, x_vlines, Scale)
    energy_plot(ST, name, fp, colors_std, x_vlines, Scale)
    modesplit_plot(ST, name, fp, colors_std, x_vlines, Scale)
    PMT_mode_plot(ST, name, fp, colors_std, x_vlines, Scale)
    VMT_fuel_plot(ST, name, fp, colors_std, x_vlines, Scale)
    PMT_vehicle_plot(ST, name, fp, colors_std, x_vlines, Scale)
    PHT_vehicle_plot(ST, name, fp, colors_std, x_vlines, Scale)

    

In [3]:
def plot(colors,# list of colors to be used (longer as the labels, or more)
         values,# matrix with -length of labels- rows per -number of scenarios- colums
         labels,# labels for the legend
         filepath_res,# where to save the figures
         name_fig,# name of the figure
         ylabel,# label of y ax
         x_vlines,#list of vlines coordinates
         min_y,# minimum y value
         max_y,# maximum y value
         labelScenarios,# name of scenarios
         is_top_labels,# print the total value and variation with baselina at the top of the column,
         title
        ):

    current_bottom = np.zeros((len(labelScenarios)))+10.0

    fig, (ax,ax0) = plt.subplots(2,1,sharex=True,gridspec_kw={'height_ratios':[10, 0.5]},
                    figsize=(np.max([2*len(labelScenarios),8]), 8))
    
    plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0, 
                        hspace=0.05)
    first = 1
    for row, key, color in zip(values, labels, colors):
        if first == 1:
            p0 = ax0.bar(labelScenarios, np.ones((len(labelScenarios))), color = color)
            if np.sum(row)>0:
                p1 = ax.bar(labelScenarios, row-10.0, color = color, bottom = current_bottom, label = key)
                fir = 1
                _labels=[]
                for y,t in zip(row, row/row[0]*100-100):
                    if fir ==1:
                        _labels.append('%.2f'%(y))
                    else:
                        if t>0:
                            _labels.append('%.2f'%(y))
                        else:
                            _labels.append('%.2f'%(y))
                    fir = 0

                ax0.bar_label(p0,labels = _labels , label_type='center', fontsize=12.5)
            current_bottom+=row-10.0
        else:
            if np.sum(row)>0:
                p1 = ax.bar(labelScenarios, row, color = color, bottom = current_bottom, label = key)
                fir = 1
                _labels=[]
                for y,t in zip(row, row/row[0]*100-100):
                    if fir ==1:
                        _labels.append('%.2f'%(y))
                    else:
                        if t>0:
                            _labels.append('%.2f'%(y))
                        else:
                            _labels.append('%.2f'%(y))
                    fir = 0

                ax.bar_label(p1,labels = _labels , label_type='center', fontsize=12.5)
            current_bottom+=row
        first = 0

    fir =1
    __labels = []
    tot_bas =np.sum(values[:,0])
    for row in values.transpose():
        if fir == 1:
            __labels.append('%.2f'%(np.sum(row)))
        else:
            t =np.sum(row)/tot_bas*100.0-100.
            if t>0:
                __labels.append('%.2f \n (+%.2f%%)'%(np.sum(row),t),)
            else:
                __labels.append('%.2f \n (%.2f%%)'%(np.sum(row),t), )
        fir = 0
    if is_top_labels:
        ax.bar_label(p1,labels = __labels, padding = 3, fontsize=12.5 )

    ax.set_ylim([min_y,max_y])
    ax.set_ylabel(ylabel, fontsize=12.5)
    y_labels = ax.get_yticks()
    ax.vlines(x=x_vlines, ymin=9, ymax=max_y, color='k',alpha = 0.4)
    ax.set_yticklabels(['%i'%(y) for y in y_labels], fontsize=12.5)
    ax.legend(loc = 9, ncol = 9, fontsize=9.8)
    ax.set_title(title)
    ax.grid(axis='y')
    ax0.set_ylim(0,1)
    ax0.set_yticklabels([0], fontsize=12.5)
    ax0.set_xticklabels(labelScenarios, fontsize=12.5)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='x',which='both',bottom=False)
    ax0.spines['top'].set_visible(False)
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)
    d = .005  
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)      
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)
    kwargs.update(transform=ax0.transAxes)  
    ax0.plot((-d, +d), (1 - d*20, 1 + d*20), **kwargs)  
    ax0.plot((1 - d, 1 + d), (1 - d*20, 1 + d*20), **kwargs)
    ax.hlines(y = min_y,xmin = -0.5, xmax = len(labelScenarios) - 0.5,color = 'k', linestyle = ':',linewidth = 2.5)
    ax0.hlines(y = 1, xmin = -0.5, xmax = len(labelScenarios) - 0.5,color = 'k', linestyle = ':',linewidth = 2.5)
    ax0.vlines(x=x_vlines, ymin=0, ymax=1, color='k',alpha = 0.4)

    plt.savefig(filepath_res + name_fig)

In [4]:
def plot2(colors,# list of colors to be used (longer as the labels, or more)
         values,# matrix with -length of labels- rows per -number of scenarios- colums
         labels,# labels for the legend
         filepath_res,# where to save the figures
         name_fig,# name of the figure
         ylabel,# label of y ax
#          x_vlines,#list of vlines coordinates
         min_y,# minimum y value
         max_y,# maximum y value
         labelScenarios,# name of scenarios
         is_top_labels,# print the total value and variation with baselina at the top of the column
          title
        ):

    current_bottom = np.zeros((len(labelScenarios)))

    fig, (ax) = plt.subplots(1,1,sharex=True,
                    figsize=(np.max([2*len(labelScenarios),8]), 8))
    
    plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0, 
                        hspace=0.05)
    for row, key, color in zip(values, labels, colors):

        if np.sum(row)>0:
            p1 = ax.bar(labelScenarios, row, color = color, bottom = current_bottom, label = key)
            fir = 1
            _labels=[]
            for y,t in zip(row, row/row[0]*100-100):
                if fir ==1:
                    _labels.append('%.2f'%(y))
                else:
                    if t>0:
                        _labels.append('%.2f'%(y))
                    else:
                        _labels.append('%.2f'%(y))
                fir = 0

            ax.bar_label(p1,labels = _labels , label_type='center', fontsize=12.5)
        current_bottom+=row

    fir =1
    __labels = []
    tot_bas =np.sum(values[:,0])
    for row in values.transpose():
        if fir == 1:
            __labels.append('%.2f'%(np.sum(row)))
        else:
            t =np.sum(row)/tot_bas*100.0-100.
            if t>0:
                __labels.append('%.2f \n (+%.2f%%)'%(np.sum(row),t),)
            else:
                __labels.append('%.2f \n (%.2f%%)'%(np.sum(row),t), )
        fir = 0
    if is_top_labels:
        ax.bar_label(p1,labels = __labels, padding = 3, fontsize=12.5 )

    ax.set_ylim([min_y,max_y])
    ax.set_ylabel(ylabel, fontsize=12.5)
    y_labels = ax.get_yticks()
#     ax.vlines(x=x_vlines, ymin=0, ymax=max_y, color='k',alpha = 0.4)
    ax.set_yticklabels(['%i'%(y) for y in y_labels], fontsize=12.5)
    ax.legend(loc = 9, ncol = 9, fontsize=9.8)
    ax.set_title(title)
    ax.grid(axis='y')
#     ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='x',which='both',bottom=False)
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)
#     d = .005  
#     kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
#     ax.plot((-d, +d), (-d, +d), **kwargs)      
#     ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)


    plt.savefig(filepath_res + name_fig)

In [5]:
def plot3(colors,# list of colors to be used (longer as the labels, or more)
         values,# matrix with -length of labels- rows per -number of scenarios- colums
         labels,# labels for the legend
         filepath_res,# where to save the figures
         name_fig,# name of the figure
         ylabel,# label of y ax
#          x_vlines,#list of vlines coordinates
         min_y,# minimum y value
         max_y,# maximum y value
         labelScenarios,# name of scenarios
         is_top_labels,# print the total value and variation with baselina at the top of the column
          title
        ):

#     current_bottom = np.zeros((len(labelScenarios)))

    fig, (ax) = plt.subplots(1,1,sharex=True,
                    figsize=(np.max([2*len(labelScenarios),8]), 8))
    
    plt.subplots_adjust(left=0.1,
                        bottom=0.1, 
                        right=0.9, 
                        top=0.9, 
                        wspace=0, 
                        hspace=0.05)
    for row, key, color in zip(values, labels, colors):

        if np.sum(row)>0:
            p1 = ax.plot(labelScenarios, row, 'o-', color = color, 
#                         bottom = current_bottom, 
                        label = key,
                        linewidth=3.7,
                        )
            fir = 1
            _labels=[]
            for y,t in zip(row, row/row[0]*100-100):
                if fir ==1:
                    _labels.append('%.2f'%(y))
                else:
                    if t>0:
                        _labels.append('%.2f'%(y))
                    else:
                        _labels.append('%.2f'%(y))
                fir = 0

#             ax.bar_label(p1,labels = _labels , label_type='center', fontsize=12.5)
#         current_bottom+=row

    fir =1
    __labels = []
    tot_bas =np.sum(values[:,0])
    for row in values.transpose():
        if fir == 1:
            __labels.append('%.2f'%(np.sum(row)))
        else:
            t =np.sum(row)/tot_bas*100.0-100.
            if t>0:
                __labels.append('%.2f \n (+%.2f%%)'%(np.sum(row),t),)
            else:
                __labels.append('%.2f \n (%.2f%%)'%(np.sum(row),t), )
        fir = 0
#     if is_top_labels:
#         ax.bar_label(p1,labels = __labels, padding = 3, fontsize=12.5 )

    ax.set_ylim([min_y,max_y])
    ax.set_ylabel(ylabel, fontsize=12.5)
    y_labels = ax.get_yticks()
#     ax.vlines(x=x_vlines, ymin=0, ymax=max_y, color='k',alpha = 0.4)
    ax.set_yticklabels(['%i'%(y) for y in y_labels], fontsize=12.5)
    ax.legend(loc = 9, ncol = 9, fontsize=9.8)
    ax.set_title(title)
    ax.grid(axis='y')
    ax.grid(axis='x')

#     ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='x',which='both',bottom=False)
    for tick in ax.get_xticklabels():
        tick.set_rotation(0)
#     d = .005  
#     kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
#     ax.plot((-d, +d), (-d, +d), **kwargs)      
#     ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)


    plt.savefig(filepath_res + name_fig)

In [6]:
#VMT
def VMT_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('VMT plot')
    RH_feature1 = ['Length Vehicle SUM Ride Hail [km]']
    RH_feature2 = ['Length Vehicle SUM Empty Ride Hail [km]']
    RH_label = 'Ride Hail Empty'
    colors = colors_std

    if is_WC:
        RH_feature1 = ['Length Vehicle SUM Empty Ride Hail [km]']+['Length Vehicle SUM Ride Hail [km]']
        RH_feature2 = ['Length Vehicle SUM Empty Ride Hail WC [km]']+['Length Vehicle SUM Ride Hail WC [km]']
        RH_label = 'Ride Hail WC'
        colors = ['#FF7E72', 
            '#FFAC91',
            '#d19f9f',
            '#5DC7B6', 
    #         '#79DFCE',
            '#8CD7FF',
            '#69ADF1']

    name_fig = 'VTA-mode.png'
    rows_VMT = [['Length Vehicle SUM Car [km]',
                    'Length Vehicle SUM Car HOV2 [km]',
                    'Length Vehicle SUM Car HOV3 [km]',
                    'Length Vehicle SUM CAV [km]'
                    ],RH_feature1
                ,RH_feature2,
                ['Length Vehicle SUM Bus [km]',
                    'Length Vehicle SUM Tram [km]',
                    'Length Vehicle SUM Rail [km]',
                    'Length Vehicle SUM Subway [km]',
                    'Length Vehicle SUM Cable Car [km]',
                    'Length Vehicle SUM Ferry [km]',
                    'Length Vehicle SUM Empty Bus [km]',
                    'Length Vehicle SUM Empty Tram [km]',
                    'Length Vehicle SUM Empty Subway [km]',
                    'Length Vehicle SUM Empty Cable Car [km]',
                    'Length Vehicle SUM Empty Ferry [km]',
                    'Length Vehicle SUM Empty Rail [km]']
                ]
    labels = ['Personal Vehicle', 'Ride Hail ', RH_label, 'Transit' ]
    title = 'Vehicle Miles Traveled, by Vehicle Type (10E+6) - Differences with the Baseline Scenario'
    ylabel = 'Vehicle Miles Traveled, by Vehicle Type (10E+6)'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000000*0.621371
    print(values)
    values = np.nan_to_num(values)
    values = values/Scale
    min_y = 1000*Scale
    max_y = np.sum(values[:,0])*1.1
    print(min_y,max_y, values)
    is_top_labels = True
    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)


In [7]:
#VHT
def VHT_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('VHT plot')

    RH_feature1 = ['Duration Vehicle SUM Ride Hail [h]']
    RH_feature2 = ['Duration Vehicle SUM Empty Ride Hail [h]']
    RH_label = 'Ride Hail Empty'
    colors = colors_std
    if is_WC:
        RH_feature1 = ['Duration Vehicle SUM Empty Ride Hail [h]']+['Duration Vehicle SUM Ride Hail [h]']
        RH_feature2 = ['Duration Vehicle SUM Empty Ride Hail WC [h]']+['Duration Vehicle SUM Ride Hail WC [h]']
        RH_label = 'Ride Hail WC'
        colors = ['#FF7E72', 
            '#FFAC91',
            '#d19f9f',
            '#5DC7B6', 
    #         '#79DFCE',
            '#8CD7FF',
            '#69ADF1']

    name_fig = 'VHT_mode.png'
    rows_VMT = [['Duration Vehicle SUM Car [h]',
                    'Duration Vehicle SUM Car HOV2 [h]',
                    'Duration Vehicle SUM Car HOV3 [h]',
                    'Duration Vehicle SUM CAV [h]'
                    ],RH_feature1
                ,RH_feature2,
                ['Duration Vehicle SUM Bus [h]',
                    'Duration Vehicle SUM Tram [h]',
                    'Duration Vehicle SUM Rail [h]',
                    'Duration Vehicle SUM Subway [h]',
                    'Duration Vehicle SUM Cable Car [h]',
                    'Duration Vehicle SUM Ferry [h]',
                    'Duration Vehicle SUM Empty Bus [h]',
                    'Duration Vehicle SUM Empty Tram [h]',
                    'Duration Vehicle SUM Empty Subway [h]',
                    'Duration Vehicle SUM Empty Cable Car [h]',
                    'Duration Vehicle SUM Empty Ferry [h]',
                    'Duration Vehicle SUM Empty Rail [h]']
                ]
    labels = ['Personal Vehicle', 'Ride Hail ', RH_label, 'Transit', ]
    title = 'Vehicle Hours Traveled, by Vehicle Type (10E+3) - Differences with the Baseline Scenario'
    ylabel = 'Vehicle Hours Traveled, by Vehicle Type (10E+3)'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000
    print(values)

    values = np.nan_to_num(values)
    values = values/Scale

    min_y = 20000*Scale
    max_y = np.sum(values[:,0])*1.15
    is_top_labels = True

    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)


In [8]:
#Speed
def speed_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('Speed plot')

    name_fig = 'Speeds_mode.png'
    RH_feature1 = ['Length Vehicle SUM Ride Hail [km]']
    RH_feature2 = ['Length Vehicle SUM Empty Ride Hail [km]']
    RH_label = 'Ride Hail Empty'
    colors = colors_std

    if is_WC:
        RH_feature1 = ['Length Vehicle SUM Ride Hail WC [km]']+['Length Vehicle SUM Ride Hail [km]']
        RH_feature2 = ['Length Vehicle SUM Empty Ride Hail WC [km]']+['Length Vehicle SUM Empty Ride Hail [km]']
        RH_label = 'Ride Hail WC'
        colors = ['#FF7E72', 
            '#FFAC91',
            '#d19f9f',
            '#5DC7B6', 
    #         '#79DFCE',
            '#8CD7FF',
            '#69ADF1']
    rows_VMT = [['Length Vehicle SUM Car [km]',
                    'Length Vehicle SUM Car HOV2 [km]',
                    'Length Vehicle SUM Car HOV3 [km]',
                    'Length Vehicle SUM CAV [km]'
                    ],RH_feature1
                ,RH_feature2,
                ['Length Vehicle SUM Bus [km]',
                    'Length Vehicle SUM Tram [km]',
                    'Length Vehicle SUM Rail [km]',
                    'Length Vehicle SUM Subway [km]',
                    'Length Vehicle SUM Cable Car [km]',
                    'Length Vehicle SUM Ferry [km]',
                    'Length Vehicle SUM Empty Bus [km]',
                    'Length Vehicle SUM Empty Tram [km]',
                    'Length Vehicle SUM Empty Subway [km]',
                    'Length Vehicle SUM Empty Cable Car [km]',
                    'Length Vehicle SUM Empty Ferry [km]',
                    'Length Vehicle SUM Empty Rail [km]']  ,          
                ['Length Vehicle SUM Walk [km]'],
                ['Length Vehicle SUM Bike [km]'],
                ]


    RH_feature1 = ['Duration Vehicle SUM Ride Hail [h]']
    RH_feature2 = ['Duration Vehicle SUM Empty Ride Hail [h]']
    if is_WC:
        RH_feature1 = ['Duration Vehicle SUM Ride Hail WC [h]']+['Duration Vehicle SUM Ride Hail [h]']
        RH_feature2 = ['Duration Vehicle SUM Empty Ride Hail WC [h]']+['Duration Vehicle SUM Empty Ride Hail [h]']

    rows_VHT = [['Duration Vehicle SUM Car [h]',
                    'Duration Vehicle SUM Car HOV2 [h]',
                    'Duration Vehicle SUM Car HOV3 [h]',
                    'Duration Vehicle SUM CAV [h]'
                    ],RH_feature1
                ,RH_feature2,
                ['Duration Vehicle SUM Bus [h]',
                    'Duration Vehicle SUM Tram [h]',
                    'Duration Vehicle SUM Rail [h]',
                    'Duration Vehicle SUM Subway [h]',
                    'Duration Vehicle SUM Cable Car [h]',
                    'Duration Vehicle SUM Ferry [h]',
                    'Duration Vehicle SUM Empty Bus [h]',
                    'Duration Vehicle SUM Empty Tram [h]',
                    'Duration Vehicle SUM Empty Subway [h]',
                    'Duration Vehicle SUM Empty Cable Car [h]',
                    'Duration Vehicle SUM Empty Ferry [h]',
                    'Duration Vehicle SUM Empty Rail [h]']  ,          
                ['Duration Vehicle SUM Walk [h]'],
                ['Duration Vehicle SUM Bike [h]'],
                ]

    labels = ['Personal Vehicle', 'Ride Hail ', RH_label, 'Transit', 'Walk', 'Bike']
    title = 'Average travel speed [km/h], by Vehicle Type - Differences with the Baseline Scenario'
    ylabel = 'Average travel speed [km/h], by Vehicle Type '
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for vhts, j, vmts in zip(rows_VHT, range(len(rows_VMT)),rows_VMT ):
            tot_vmt = 0
            for vht, vmt in zip(vhts,vmts):
                if float(summarytable.at[vht, scenario])>0:
                    values[j,i] += float(summarytable.at[vmt, scenario])**2/float(summarytable.at[vht, scenario])
                    tot_vmt +=float(summarytable.at[vmt, scenario])
            if tot_vmt>0:
                values[j,i] /=tot_vmt
            else:
                values[j,i] = 0
    print(values)

    values = np.nan_to_num(values)
    min_y = 0
    max_y = np.max(values[:,0])*1.2
    is_top_labels = True

    plot3(colors, values, labels, filepath_res, name_fig, ylabel, min_y, max_y, nameScenarios, is_top_labels, title)


In [9]:
#Energy
def energy_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('Energy plot')

    RH_feature1 = ['Energy Vehicle SUM Ride Hail [GJ]']
    RH_feature2 = ['Energy Vehicle SUM Empty Ride Hail [GJ]']
    RH_label = 'Ride Hail Empty'
    colors = colors_std
    if is_WC:
        RH_feature1 = ['Energy Vehicle SUM Empty Ride Hail [GJ]']+['Energy Vehicle SUM Ride Hail [GJ]']
        RH_feature2 = ['Energy Vehicle SUM Empty Ride Hail WC [GJ]']+['Energy Vehicle SUM Ride Hail WC [GJ]']
        RH_label = 'Ride Hail WC'
        colors = ['#FF7E72', 
            '#FFAC91',
            '#d19f9f',
            '#5DC7B6', 
    #         '#79DFCE',
            '#8CD7FF',
            '#69ADF1']

    name_fig = 'Energy_mode.png'
    rows_VMT = [['Energy Vehicle SUM Car [GJ]',
                    'Energy Vehicle SUM Car HOV2 [GJ]',
                    'Energy Vehicle SUM Car HOV3 [GJ]',
                    'Energy Vehicle SUM CAV [GJ]'
                    ],RH_feature1
                ,RH_feature2,
                ['Energy Vehicle SUM Bus [GJ]',
                    'Energy Vehicle SUM Tram [GJ]',
                    'Energy Vehicle SUM Rail [GJ]',
                    'Energy Vehicle SUM Subway [GJ]',
                    'Energy Vehicle SUM Cable Car [GJ]',
                    'Energy Vehicle SUM Ferry [GJ]',
                   'Energy Vehicle SUM Empty Bus [GJ]',
                    'Energy Vehicle SUM Empty Tram [GJ]',
                    'Energy Vehicle SUM Empty Subway [GJ]',
                    'Energy Vehicle SUM Empty Cable Car [GJ]',
                    'Energy Vehicle SUM Empty Ferry [GJ]',
                    'Energy Vehicle SUM Empty Rail [GJ]']
                ]
    labels = ['Personal Vehicle', 'Ride Hail ', RH_label, 'Transit',]
    title = 'Energy Consumption, by Vehicle Type [TJ] - Differences with the Baseline Scenario'
    ylabel = 'Energy Consumption, by Vehicle Type [TJ], by Vehicle Type'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000
    print(values)
    colors = colors_std
    values = np.nan_to_num(values)
    values = values/Scale

    min_y = 2500*Scale
    max_y = np.sum(values[:,0])*1.15
    is_top_labels = True

    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)

In [10]:
#Mode Split
def modesplit_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('modesplit plot')

    name_fig = 'Mode split.png'
    rows_VMT = [['Trip Exec Share Car',
                    'Trip Exec Share HOV3 Passenger',
                    'Trip Exec Share HOV2 Passenger',
                    'Trip Exec Share HOV2 Driver',
                    'Trip Exec Share HOV3 Driver'
                    ],
                ['Trip Exec Share Ride Hail'],
                    ['Trip Exec Share Ride Hail Pooled'],
                ['Trip Exec Share Walk-Transit',
                    'Trip Exec Share Ride Hail-Transit',
                    'Trip Exec Share Drive-Transit',
                    'Trip Exec Share Cable Car',
                    'Trip Exec Share Bike-Transit',
                    'Trip Exec Share Bus',
                    'Trip Exec Share Subway',
                    'Trip Exec Share Tram',
                    'Trip Exec Share Rail'
                ],
                ['Trip Exec Share Walk'],
                ['Trip Exec Share Bike']]




    labels = ['Personal Vehicle','Ride Hail','Ride Hail Pooled','Transit', 'Walk', 'Bike',]
    title = 'Trip Mode Split [%] - Differences with the Baseline Scenario'
    ylabel = 'Mode Split [%]'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])*100
    print(values)
    values = np.nan_to_num(values)
    colors = colors_std

    min_y = 80
    max_y = np.sum(values[:,0])*1.05
    is_top_labels = False
    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)

In [11]:
#PMT -mode
def PMT_mode_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('PMT plot')

    name_fig = 'PMT_mode.png'
    rows_VMT = [['Length Mode SUM Car [km]',
                    'Length Mode SUM HOV3 Passenger [km]',
                    'Length Mode SUM HOV2 Passenger [km]',
                    'Length Mode SUM HOV2 Driver [km]',
                    'Length Mode SUM HOV3 Driver [km]'
                    ],
                ['Length Mode SUM Ride Hail [km]'],
                    ['Length Mode SUM Ride Hail Pooled [km]'],
                ['Length Mode SUM Walk-Transit [km]',
                    'Length Mode SUM Ride Hail-Transit [km]',
                    'Length Mode SUM Drive-Transit [km]',
                    'Length Mode SUM Cable Car [km]',
                    'Length Mode SUM Bike-Transit [km]',
                    'Length Mode SUM Bus [km]',
                    'Length Mode SUM Subway [km]',
                    'Length Mode SUM Tram [km]',
                    'Length Mode SUM Rail [km]'
                ],
                ['Length Mode SUM Walk [km]'],
                ['Length Mode SUM Bike [km]']]




    labels = ['Personal Vehicle','Ride Hail','Ride Hail Pooled','Transit', 'Walk', 'Bike',]
    title = 'Person Miles Traveled, by Transport Mode (10E+6) - Differences with the Baseline Scenario'
    ylabel = 'Person Miles Traveled (10E+6)'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000000
    print(values)
    colors = colors_std
    values = np.nan_to_num(values)
    values = values/Scale
    min_y = 24/Scale
    max_y = np.sum(values[:,0])*1.05
    is_top_labels = True
    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)

In [12]:
#VMT
def VMT_fuel_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('VMT - Fuel plot')

    name_fig = 'VTA_fueltype.png'
    rows_VMT = [['Length Vehicle SUM Gasoline [km]',
                    ],
                ['Length Vehicle SUM Diesel [km]'],
                ['Length Vehicle SUM Biodiesel [km]',
                    'Length Vehicle SUM Electricity [km]'
                ]]
    labels = ['Gasoline', 'Diesel','Other']
    title = 'Vehicle Miles Traveled, by Fuel Type (10E+6) - Differences with the Baseline Scenario'
    ylabel = 'Vehicle Miles Traveled, by Fuel Type (10E+6)'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000000*0.621371
    print(values)
    colors = colors_std
    values = np.nan_to_num(values)
    values = values/Scale

    min_y = 1100*Scale
    max_y = np.sum(values[:,0])*1.05
    is_top_labels = True
    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)

In [13]:
#PMT-vehicle
def PMT_vehicle_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('PMT vehicle plot')

    RH_feature1 = ['Length Person SUM Ride Hail [km]']
    RH_feature2 = ['Length Person SUM Ride Hail WC [km]']
    RH_label = 'Ride Hail WC'

    colors = ['#FF7E72', 
        '#FFAC91',
        '#d19f9f',
        '#5DC7B6', 
    #         '#79DFCE',
        '#8CD7FF',
        '#69ADF1']

    name_fig = 'PMT-vehicle.png'
    rows_VMT = [['Length Person SUM Car [km]',
                    'Length Person SUM Car HOV2 [km]',
                    'Length Person SUM Car HOV3 [km]',
                    'Length Person SUM CAV [km]'
                    ],RH_feature1
                ,RH_feature2,
                ['Length Person SUM Bus [km]',
                    'Length Person SUM Tram [km]',
                    'Length Person SUM Rail [km]',
                    'Length Person SUM Subway [km]',
                    'Length Person SUM Cable Car [km]',
                    'Length Person SUM Ferry [km]',
                    'Length Person SUM Empty Bus [km]',
                    'Length Person SUM Empty Tram [km]',
                    'Length Person SUM Empty Subway [km]',
                    'Length Person SUM Empty Cable Car [km]',
                    'Length Person SUM Empty Ferry [km]',
                    'Length Person SUM Empty Rail [km]']
                ]
    labels = ['Personal Vehicle', 'Ride Hail ', RH_label, 'Transit' ]
    title = 'Person Miles Traveled, by Vehicle Type (10E+6) - Differences with the Baseline Scenario'
    ylabel = 'Person Miles Traveled, by Vehicle Type (10E+6)'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000000*0.621371
    print(values)
    values = np.nan_to_num(values)
    values = values/Scale

    min_y = 1100*Scale
    max_y = np.sum(values[:,0])*1.05
    is_top_labels = True
    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)


In [14]:
#PHT-vehicle
def PHT_vehicle_plot(summarytable, nameScenarios, filepath_res, colors_std, x_vlines, Scale):
    print('PHT vehicle plot')

    RH_feature1 = ['Duration Person SUM Ride Hail [h]']
    RH_feature2 = ['Duration Person SUM Ride Hail WC [h]']
    RH_label = 'Ride Hail WC'

    colors = ['#FF7E72', 
        '#FFAC91',
        '#d19f9f',
        '#5DC7B6', 
    #         '#79DFCE',
        '#8CD7FF',
        '#69ADF1']

    name_fig = 'PHT-vehicle.png'
    rows_VMT = [['Duration Person SUM Car [h]',
                    'Duration Person SUM Car HOV2 [h]',
                    'Duration Person SUM Car HOV3 [h]',
                    'Duration Person SUM CAV [h]'
                    ],RH_feature1
                ,RH_feature2,
                ['Duration Person SUM Bus [h]',
                    'Duration Person SUM Tram [h]',
                    'Duration Person SUM Rail [h]',
                    'Duration Person SUM Subway [h]',
                    'Duration Person SUM Cable Car [h]',
                    'Duration Person SUM Ferry [h]',
                    'Duration Person SUM Empty Bus [h]',
                    'Duration Person SUM Empty Tram [h]',
                    'Duration Person SUM Empty Subway [h]',
                    'Duration Person SUM Empty Cable Car [h]',
                    'Duration Person SUM Empty Ferry [h]',
                    'Duration Person SUM Empty Rail [h]']
                ]
    labels = ['Personal Vehicle', 'Ride Hail ', RH_label, 'Transit' ]
    title = 'Person Hours Traveled, by Vehicle Type (10E+3) - Differences with the Baseline Scenario'
    ylabel = 'Person Hours Traveled, by Vehicle Type (10E+3)'
    values = np.zeros((len(rows_VMT),len(nameScenarios)))

    for scenario, i in zip(nameScenarios, range(len(nameScenarios))):
        for attributes, j in zip(rows_VMT, range(len(rows_VMT))):
            for attribute in attributes:
                values[j,i] += float(summarytable.at[attribute, scenario])/1000
    print(values)
    values = np.nan_to_num(values)
    values = values/Scale
    min_y = 30000*Scale
    max_y = np.sum(values[:,0])*1.05
    is_top_labels = True
    plot(colors, values, labels, filepath_res, name_fig, ylabel, x_vlines, min_y, max_y, nameScenarios, is_top_labels, title)
